### So you want to use AWS ec2 instance with GPU to train your models....
Before you jump into using SageMaker ( a Docker based solution ), get yourself more familiar with using ec2

Assumption is that either you have access to AWS service or can ask someone else to set it up for you

The intent is to setup a Jupyter Notebook instance on AWS ec2, that you can access from your local laptop and create appropriate model. You will use t2.micro instance to check for valid syntax and valid function of the model. (less than 50 cents per day )

Once you feel comfortable, you can create an ec2 instance with 1 GPU to verify the same using semi-long running training duration. ( about $22 per day )

Finally, when you are really comfortable with the whole, follow SageMaker process to train your model on multi-GPU environment (if required)


#### Create (or Launch) a new ec2 instance: t2.micro

Select the instance that has python installed
![ec2](ec2.png)

Choose the t2.micro instance
![t2micro](t2micro.png)

Allow SSH and ports to be accessible
![ports](ports.png)

#### If using Windows to access ec2, you MUST use putty to convert "key" to putty private key

Launch Putty Key Generator and read in the AWS generate key and save to putty private key
![keygen](keygen.png)

Save the private key. Let's assume it's mac.ppk


#### setup the connection info in Putty

When you try to connect to ec2 instance, you will see the ssh command something like:

**ssh -i "aws.key.pem" ec2-user@somehost.compute.amazonaws.com**

You will be using the ec2-user@somehost.compute.amazonaws.com value into Putty
![connect](connect.png)

Click Connection/SSH/Auth and read in the private key, for example: mac.ppk
![loadkehy](loadkey.png)

You will now setup tunnel to ec2 instance.
Click Tunnel and enter:
```
source port = 8888
destination = ec2-user@host.compute.amazonaws.com:8887
```
and Click Add
![tunnel](tunnel.png)

You can save the profile or click Open to get connected to ec2 instance.




#### install Python 3.6 on ec2 instance
Once you are connected execute `sudo yum list | grep python3`

You will see a listing of `python36-<MODULE>.x86_64` Install all of them except the modules that end with `.686` and `debug` by executing `sudo yum install x y z` where x y z are the names of the modules

You will be asked `Y/n` to continue. Hit `Y`

After the installation completed, verify the version by executing `python3 --version`.  You should see an output 3.8.* where * is the latest version



#### install pip
Install pip by executing below:
```
mkdir stuff
cd stuff
curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
python3 ./get-pip.py --user
```


#### install python modules using pip
Create a file `requirement.txt` with the following content:
```
absl-py >= 0.7.0
astor >= 0.6.0
gast >= 0.2.0
google_pasta >= 0.1.2
keras_applications >= 1.0.6
keras_preprocessing >= 1.0.5
numpy >= 1.14.5, < 2.0
six >= 1.10.0
protobuf >= 3.6.1
tensorboard >= 1.13.0, < 1.14.0
tensorflow_estimator >= 1.13.0rc0, < 1.14.0rc0
termcolor >= 1.1.0
urllib3
pyYAML
scipy
matplotlib
pillow
simpy
nose
pydot
pandas
tensorflow==1.13.1
jupyter
notebook
keras
```
Execute command: `python3 -m pip -r requirement.txt --user` and the modules listed will get downloaded

#### configure Jupyter Notebook
execute below to configure Jupyter Notebook to be accessible from your local laptop.
Basically generate and open up a config file:
```
jupyter notebook --generate-config
cd ~/.jupyter
vi jupyter_notebook_config.py
```
Uncomment and set values for these two lines:
```
c.NotebookApp.allow_origin = '*'
c.NotebookApp.ip = '0.0.0.0'
```
And save the file

#### launch Jupyter Notebook in non-browser mode
launch jupyter notebook by executing:
```
jupyter notebook --no-browser --ip=0.0.0.0 --port=8887
```
You will see it launch. Note the `/?token=xyz...`
![terminal](terminal.png)


#### Connect to the AWS jupyter notebook from you local laptop
Launch your browser and enter:
```
http://localhost:8888/?token=xyz...`
```
The value to enter for token should match what you see in the ec2 terminal

You will now see Jupyter Notebook running on ec2
![ec2 terminal](jup.png)


### to transfer files between your laptop and ec2 instance, use psftp
Use the following command to get connected to the ec2 instance via sftp using Putty
```
psftp -i <thekeyfile> ec2-user@somehost.compute.amazonaws.com
```

#### when done with Jupyter Notebook, at least stop the ec2 instance (or teminate if will not use anymore)